In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import keras.backend as K
import tensorflow as tf

Using TensorFlow backend.


In [2]:
X_train = np.random.randint(3, high=16, size=(11000,8))
Y_train = np.sort(X_train, axis=-1)
X_target = np.copy(Y_train)
X_target[:,1:] = X_target[:,:-1]
X_target[:,0] = 1
X_train = np.concatenate([X_train, X_target], axis=-1)
X_train,X_test = X_train[1000:],X_train[:1000]
Y_train,Y_test = Y_train[1000:],Y_train[:1000]
X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

((10000, 16), (10000, 8), (1000, 16), (1000, 8))

In [3]:
data_input_dim = max(np.max(X_train), np.max(X_test)) + 1
data_input_dim

16

In [4]:
data_output_dim = max(np.max(Y_train), np.max(Y_test)) + 1
data_output_dim

16

In [5]:
class DNCCell(keras.layers.Layer):
    
    def __init__(self,
                 output_size,
                 word_size,
                 memory_size,
                 register_size=2,
                 write_head_count=2,
                 read_head_count=2,
                 enable_temporal=True,
                 enable_controller_output_resize=True,
                 enable_final_bias=True,
                 bypass_dropout_factor=None,
                 **kwargs):
        self.output_size = output_size
        self.word_size = word_size
        self.memory_size = memory_size
        self.register_size = register_size
        self.write_head_count = write_head_count
        self.read_head_count = read_head_count
        self.enable_temporal = enable_temporal
        self.enable_controller_output_resize = enable_controller_output_resize
        self.enable_final_bias = enable_final_bias
        self.bypass_dropout_factor = bypass_dropout_factor
        self.state_size = (output_size,)
        self.state_size += (register_size * word_size,)
        self.state_size += (register_size * word_size,)
        self.state_size += (memory_size * word_size,)
        self.state_size += (read_head_count * memory_size,)
        self.state_size += (read_head_count * word_size,)
        self.state_size += (write_head_count * memory_size,)
        self.state_size += (memory_size,)
        if enable_temporal:
            self.state_size += (write_head_count * memory_size,)
            self.state_size += (write_head_count * memory_size**2,)
        super().__init__(**kwargs)
    
    def build(self, input_shape):
        read_vec_size = self.word_size * self.read_head_count
        controller_input_size = input_shape[-1]
        controller_input_size += read_vec_size
        controller_input_size += self.register_size*self.word_size
        controller_kernel_size = self.register_size*self.word_size*4
        controller_hidden_size = self.register_size*self.word_size
        interface_vec_size = read_vec_size
        interface_vec_size += self.read_head_count*2
        if self.enable_temporal:
            interface_vec_size += self.read_head_count*3
        interface_vec_size += self.write_head_count*self.word_size*3
        interface_vec_size += self.write_head_count*3
        self.kernel_controller_hidden = self.add_weight(
            shape=(controller_input_size, controller_kernel_size),
            initializer='glorot_normal', name='kernel_controller_hidden')
        self.bias_controller_hidden = self.add_weight(
            shape=(controller_kernel_size,),
            initializer='zeros', name='bias_controller_hidden')
        self.kernel_controller_output = self.add_weight(
            shape=(controller_hidden_size, self.output_size + interface_vec_size),
            initializer='glorot_normal', name='kernel_controller_output')
        self.kernel_read_vec_to_output = self.add_weight(
            shape=(read_vec_size, self.output_size),
            initializer='glorot_normal', name='kernel_read_vec_to_output')
        if self.enable_controller_output_resize:
            self.scalar_controller_output_resize = self.add_weight(
                shape=(self.output_size,),
                initializer='ones', name='scalar_controller_output_resize')
        if self.enable_final_bias:
            self.bias_final_output = self.add_weight(
                shape=(self.output_size,),
                initializer='zeros', name='bias_final_output')
        super().build(input_shape)
    
    def call(self, inputs, states, training=None):
        
        def oneplus(x):
            return K.softplus(x) + 1.
        
        def similarity(m, k, b):
            dot = K.batch_dot(k, m, axes=2)
            m_len = K.sqrt(K.sum(K.square(m), axis=-1))
            k_len = K.sqrt(K.sum(K.square(k), axis=-1))
            mk_len = K.expand_dims(k_len, axis=-1) @ K.expand_dims(m_len, axis=-2)
            mk_len = K.switch(K.not_equal(mk_len, 0.), mk_len, K.ones_like(mk_len))
            cos_sim = dot / mk_len
            return K.softmax(cos_sim * K.expand_dims(b))
        
        def batch_invert_permutation(permutations):
            # from https://github.com/deepmind/dnc/blob/master/dnc/util.py
            perm = tf.cast(permutations, tf.float32)
            dim = int(perm.get_shape()[-1])
            size = tf.cast(tf.shape(perm)[0], tf.float32)
            delta = tf.cast(tf.shape(perm)[-1], tf.float32)
            rg = tf.range(0, size * delta, delta, dtype=tf.float32)
            rg = tf.expand_dims(rg, 1)
            rg = tf.tile(rg, [1, dim])
            perm = tf.add(perm, rg)
            flat = tf.reshape(perm, [-1])
            perm = tf.invert_permutation(tf.cast(flat, tf.int32))
            perm = tf.reshape(perm, [-1, dim])
            return tf.subtract(perm, tf.cast(rg, tf.int32))

        def batch_gather(values, indices):
            # from https://github.com/deepmind/dnc/blob/master/dnc/util.py
            idx = tf.expand_dims(indices, -1)
            size = tf.shape(indices)[0]
            rg = tf.range(size, dtype=tf.int32)
            rg = tf.expand_dims(rg, -1)
            rg = tf.tile(rg, [1, int(indices.get_shape()[-1])])
            rg = tf.expand_dims(rg, -1)
            gidx = tf.concat([rg, idx], -1)
            return tf.gather_nd(values, gidx)
        
        _, register_s_last, register_h_last, memory_last, \
            read_weights_last, read_vec_last, write_weights_last, \
            usage_last, *temporal_states = states
        memory_last = K.reshape(memory_last,
            (-1,self.memory_size,self.word_size))
        read_weights_last = K.reshape(read_weights_last,
            (-1,self.read_head_count,self.memory_size))
        write_weights_last = K.reshape(write_weights_last,
            (-1,self.write_head_count,self.memory_size))
        if not self.enable_temporal:
            assert not temporal_states
        else:
            preced_last, link_last = temporal_states
            preced_last = K.reshape(preced_last,
                (-1,self.write_head_count,self.memory_size))
            link_last = K.reshape(link_last,
                (-1,self.write_head_count,self.memory_size,self.memory_size))
        
        # feeding controller with current input and last read vecs
        output = K.concatenate([inputs, read_vec_last, register_h_last])
        output = output @ self.kernel_controller_hidden
        output = output + self.bias_controller_hidden
        ctr_input_gate = output[:,:self.register_size*self.word_size]
        ctr_input_gate = K.sigmoid(ctr_input_gate)
        ctr_forget_gate = output[:,
            self.register_size*self.word_size:
            self.register_size*self.word_size*2]
        ctr_forget_gate = K.sigmoid(ctr_forget_gate)
        register_s = output[:,
            self.register_size*self.word_size*2:
            self.register_size*self.word_size*3]
        register_s = ctr_input_gate * K.tanh(register_s)
        register_s = register_s + ctr_forget_gate*register_s_last
        ctr_output_gate = output[:,
            self.register_size*self.word_size*3:
            self.register_size*self.word_size*4]
        ctr_output_gate = K.sigmoid(ctr_output_gate)
        register_h = ctr_output_gate * K.tanh(register_s)
        output = register_h @ self.kernel_controller_output
        
        # break down controller output into semi final output and interface vec
        interface_vec = output[:,self.output_size:]
        output = output[:,:self.output_size]
        if self.bypass_dropout_factor is not None:
            output = K.in_train_phase(K.dropout(
                output,self.bypass_dropout_factor),output,training=training)
        if self.enable_controller_output_resize:
            output = output * self.scalar_controller_output_resize
        
        # break down interface vec
        interface_pos_last = 0
        interface_partition = []
        for interface_part_len in [
            self.read_head_count * self.word_size,
            self.read_head_count,
            self.write_head_count * self.word_size,
            self.write_head_count,
            self.write_head_count * self.word_size,
            self.write_head_count * self.word_size,
            self.read_head_count,
            self.write_head_count,
            self.write_head_count,
            *([self.read_head_count * 3] if self.enable_temporal else [])]:
            interface_pos = interface_pos_last + interface_part_len
            interface_partition.append(
                interface_vec[:,interface_pos_last:interface_pos])
            interface_pos_last = interface_pos
        read_keys, read_stre, write_keys, write_stre, erase_vecs, \
            write_vecs, free_gates, alloc_gates, write_gates, \
            *temporal_interface_partition = interface_partition
        read_keys = K.reshape(read_keys,(-1,self.read_head_count,self.word_size))
        read_stre = oneplus(read_stre)
        write_keys = K.reshape(write_keys,(-1,self.write_head_count,self.word_size))
        write_stre = oneplus(write_stre)
        erase_vecs = K.reshape(erase_vecs,(-1,self.write_head_count,self.word_size))
        erase_vecs = K.sigmoid(erase_vecs)
        write_vecs = K.reshape(write_vecs,(-1,self.write_head_count,self.word_size))
        free_gates = K.expand_dims(free_gates)
        free_gates = K.sigmoid(free_gates)
        alloc_gates = K.expand_dims(alloc_gates)
        alloc_gates = K.sigmoid(alloc_gates)
        write_gates = K.expand_dims(write_gates)
        write_gates = K.sigmoid(write_gates)
        if not self.enable_temporal:
            assert not temporal_interface_partition
        else:
            read_modes, = temporal_interface_partition
            read_modes = K.reshape(read_modes,(-1,self.read_head_count,3))
            read_modes = K.softmax(read_modes,axis=-1)
        
        # compute allocation vector
        retention = K.prod(1.-(free_gates*read_weights_last), axis=-2)
        # https://github.com/deepmind/dnc/blob/master/dnc/addressing.py
        # according to the deepmind implementation,
        # only write weight is not differentiable
        write_weights_last_nograd = K.stop_gradient(write_weights_last)
        # reduce for multi-write-head, not presented in original papaer
        write_weights_last_nograd = 1.-K.prod(1.-write_weights_last_nograd, axis=-2)
        usage = ((usage_last+write_weights_last_nograd) - \
                usage_last*write_weights_last_nograd) * retention
        # loop for multi-write-head support
        mwh_write_gates = write_gates * alloc_gates
        mwh_usage = usage
        mwh_alloc = []
        for i in range(self.write_head_count):
            # quickfix for tf.cumprod grad bug
            # https://github.com/tensorflow/tensorflow/issues/3862
            usage_qfixed = 1e-6 + (1-1e-6) * mwh_usage
            usage_asc,usage_perm = tf.nn.top_k(1.-usage_qfixed,k=self.memory_size)
            usage_asc = 1.-usage_asc
            alloc_asc = (1.-usage_asc) * tf.cumprod(usage_asc,axis=-1,exclusive=True)
            alloc_perm = batch_invert_permutation(usage_perm)
            alloc = batch_gather(alloc_asc, alloc_perm)
            mwh_alloc.append(alloc)
            mwh_usage += (1.-mwh_usage) * mwh_write_gates[:,i,:] * alloc
        alloc = K.stack(mwh_alloc, axis=-2)
        
        # compute write weight
        write_sims = similarity(memory_last,write_keys,write_stre)
        write_weights = write_gates*(alloc_gates*alloc + (1.-alloc_gates)*write_sims)
        
        # compute precedence and temporal links
        if self.enable_temporal:
            write_weights_rep_h =  K.expand_dims(write_weights,axis=-1)
            write_weights_rep_v =  K.expand_dims(write_weights,axis=-2)
            link = (1.-(write_weights_rep_h+write_weights_rep_v))*link_last + \
                write_weights_rep_h * K.expand_dims(preced_last,axis=-2)
            link = link * (1.-tf.eye(self.memory_size))
            read_weights_last_rep = K.repeat_elements(
                K.expand_dims(read_weights_last,axis=-3),self.write_head_count,-3)
            # reduce sum for multi-write-head
            link_forward = K.sum(read_weights_last_rep @ link,axis=-3)
            link_backward = K.sum(K.permute_dimensions(link @ K.permute_dimensions(
                read_weights_last_rep,(0,1,3,2)),(0,1,3,2)),axis=-3)
            preced = (1.-K.sum(
                write_weights,axis=-1,keepdims=True))*preced_last + write_weights
        
        # update memory
        m_reset = K.expand_dims(write_weights,axis=-1) @ \
            K.expand_dims(erase_vecs,axis=-2)
        # reduce prod for multi-write-head
        m_keep = K.prod(1.-m_reset,axis=-3)
        m_new = K.expand_dims(write_weights,axis=-1) @ \
            K.expand_dims(write_vecs,axis=-2)
        # reduce sum for multi-write-head
        m_new = K.sum(m_new, axis=-3)
        memory = memory_last*m_keep + m_new
        
        # compute read weights and read vectors
        read_sims = similarity(memory,read_keys,read_stre)
        if not self.enable_temporal:
            read_weights = read_sims
        else:
            read_weights = read_modes[:,:,0:1] * read_sims + \
                            read_modes[:,:,1:2] * link_forward + \
                            read_modes[:,:,2:3] * link_backward
        read_vecs = read_weights @ memory
        read_vec = K.reshape(read_vecs, (-1,self.read_head_count*self.word_size))
        
        # compute final output from controller output and current read vecs
        output = output + read_vec @ self.kernel_read_vec_to_output
        if self.enable_final_bias:
            output = output + self.bias_final_output
        
        return output, [
            output,
            register_s,
            register_h,
            K.reshape(memory, (-1,self.memory_size*self.word_size)),
            K.reshape(read_weights, (-1,self.read_head_count*self.memory_size)),
            read_vec,
            K.reshape(write_weights, (-1,self.write_head_count*self.memory_size)),
            usage,
            *([
                K.reshape(preced, (-1,self.write_head_count*self.memory_size)),
                K.reshape(link, (-1,self.write_head_count*self.memory_size**2))
            ] if self.enable_temporal else [])]
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)

In [6]:
X = X_inputs = keras.layers.Input((None,), dtype='int32')
X = keras.layers.Embedding(data_input_dim, 16, mask_zero=True)(X)
X = keras.layers.RNN(DNCCell(data_output_dim, 8, 16), return_sequences=True)(X)
X = keras.layers.Lambda(lambda x,s: x[:,s:,:],
    arguments={'s':X_train.shape[1]//2})(X)
X = keras.layers.Softmax()(X)
X = [keras.layers.Lambda(lambda x,s: x[:,s,:], name=f'o{i+1}',
    arguments={'s':i}, output_shape=(data_output_dim,))(X)
    for i in range(X_train.shape[1]//2)]
M = keras.Model(X_inputs, X)
M.compile('nadam', 'sparse_categorical_crossentropy', ['acc'])
M.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 16)     256         input_1[0][0]                    
__________________________________________________________________________________________________
rnn_1 (RNN)                     (None, None, 16)     4960        embedding_1[0][0]                
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, 16)     0           rnn_1[0][0]                      
__________________________________________________________________________________________________
softmax_1 

In [7]:
M.fit(X_train, np.split(Y_train,Y_train.shape[-1],axis=-1),
    validation_data=(X_test,np.split(Y_test,Y_test.shape[-1],axis=-1)),
    batch_size=128, epochs=100, callbacks=[
    keras.callbacks.ReduceLROnPlateau('loss', patience=3, verbose=1)
])

Train on 10000 samples, validate on 1000 samples
Epoch 1/100
10000/10000 [==============================] - 26s 3ms/step - loss: 18.7916 - o1_loss: 2.4331 - o2_loss: 2.4299 - o3_loss: 2.4456 - o4_loss: 2.4444 - o5_loss: 2.4089 - o6_loss: 2.3417 - o7_loss: 2.2272 - o8_loss: 2.0608 - o1_acc: 0.1253 - o2_acc: 0.1309 - o3_acc: 0.1583 - o4_acc: 0.1632 - o5_acc: 0.1419 - o6_acc: 0.1125 - o7_acc: 0.1458 - o8_acc: 0.4216 - val_loss: 15.6455 - val_o1_loss: 1.8693 - val_o2_loss: 1.9217 - val_o3_loss: 1.9935 - val_o4_loss: 2.0573 - val_o5_loss: 2.0784 - val_o6_loss: 2.0410 - val_o7_loss: 1.9415 - val_o8_loss: 1.7427 - val_o1_acc: 0.2410 - val_o2_acc: 0.2730 - val_o3_acc: 0.2650 - val_o4_acc: 0.3020 - val_o5_acc: 0.2370 - val_o6_acc: 0.2230 - val_o7_acc: 0.2220 - val_o8_acc: 0.3880
Epoch 2/100
10000/10000 [==============================] - 21s 2ms/step - loss: 14.2192 - o1_loss: 1.6807 - o2_loss: 1.8043 - o3_loss: 1.8668 - o4_loss: 1.8813 - o5_loss: 1.8864 - o6_loss: 1.8677 - o7_loss: 1.7508 - o8_

10000/10000 [==============================] - 21s 2ms/step - loss: 9.4494 - o1_loss: 1.0178 - o2_loss: 1.1051 - o3_loss: 1.0775 - o4_loss: 1.1347 - o5_loss: 1.1989 - o6_loss: 1.2870 - o7_loss: 1.3954 - o8_loss: 1.2330 - o1_acc: 0.5444 - o2_acc: 0.5004 - o3_acc: 0.5238 - o4_acc: 0.5054 - o5_acc: 0.4851 - o6_acc: 0.4492 - o7_acc: 0.3672 - o8_acc: 0.4537 - val_loss: 9.1129 - val_o1_loss: 1.0302 - val_o2_loss: 1.0375 - val_o3_loss: 1.0293 - val_o4_loss: 1.0897 - val_o5_loss: 1.1469 - val_o6_loss: 1.2329 - val_o7_loss: 1.3491 - val_o8_loss: 1.1974 - val_o1_acc: 0.5260 - val_o2_acc: 0.5530 - val_o3_acc: 0.5310 - val_o4_acc: 0.5120 - val_o5_acc: 0.5160 - val_o6_acc: 0.4640 - val_o7_acc: 0.3950 - val_o8_acc: 0.4360
Epoch 13/100
10000/10000 [==============================] - 21s 2ms/step - loss: 9.2655 - o1_loss: 0.9808 - o2_loss: 1.0647 - o3_loss: 1.0536 - o4_loss: 1.1115 - o5_loss: 1.1724 - o6_loss: 1.2799 - o7_loss: 1.3787 - o8_loss: 1.2240 - o1_acc: 0.5584 - o2_acc: 0.5114 - o3_acc: 0.5335

10000/10000 [==============================] - 21s 2ms/step - loss: 7.5544 - o1_loss: 0.6641 - o2_loss: 0.6913 - o3_loss: 0.7807 - o4_loss: 0.8968 - o5_loss: 1.0005 - o6_loss: 1.1279 - o7_loss: 1.2653 - o8_loss: 1.1279 - o1_acc: 0.7069 - o2_acc: 0.6894 - o3_acc: 0.6545 - o4_acc: 0.5987 - o5_acc: 0.5580 - o6_acc: 0.5034 - o7_acc: 0.4263 - o8_acc: 0.4858 - val_loss: 7.5541 - val_o1_loss: 0.6609 - val_o2_loss: 0.6661 - val_o3_loss: 0.7687 - val_o4_loss: 0.9315 - val_o5_loss: 1.0108 - val_o6_loss: 1.0965 - val_o7_loss: 1.2681 - val_o8_loss: 1.1516 - val_o1_acc: 0.7020 - val_o2_acc: 0.7050 - val_o3_acc: 0.6280 - val_o4_acc: 0.5740 - val_o5_acc: 0.5630 - val_o6_acc: 0.4910 - val_o7_acc: 0.4100 - val_o8_acc: 0.4690
Epoch 24/100
10000/10000 [==============================] - 21s 2ms/step - loss: 7.5017 - o1_loss: 0.6616 - o2_loss: 0.7018 - o3_loss: 0.7943 - o4_loss: 0.8993 - o5_loss: 0.9900 - o6_loss: 1.1046 - o7_loss: 1.2381 - o8_loss: 1.1120 - o1_acc: 0.7080 - o2_acc: 0.6793 - o3_acc: 0.6458

10000/10000 [==============================] - 23s 2ms/step - loss: 5.9749 - o1_loss: 0.5571 - o2_loss: 0.5494 - o3_loss: 0.6055 - o4_loss: 0.6951 - o5_loss: 0.7753 - o6_loss: 0.8659 - o7_loss: 0.9776 - o8_loss: 0.9489 - o1_acc: 0.7625 - o2_acc: 0.7640 - o3_acc: 0.7345 - o4_acc: 0.7015 - o5_acc: 0.6556 - o6_acc: 0.6139 - o7_acc: 0.5557 - o8_acc: 0.5601 - val_loss: 5.6580 - val_o1_loss: 0.5655 - val_o2_loss: 0.5198 - val_o3_loss: 0.5472 - val_o4_loss: 0.6517 - val_o5_loss: 0.7288 - val_o6_loss: 0.7864 - val_o7_loss: 0.9222 - val_o8_loss: 0.9364 - val_o1_acc: 0.7610 - val_o2_acc: 0.7810 - val_o3_acc: 0.7630 - val_o4_acc: 0.7040 - val_o5_acc: 0.6630 - val_o6_acc: 0.6500 - val_o7_acc: 0.5690 - val_o8_acc: 0.5500
Epoch 35/100
10000/10000 [==============================] - 22s 2ms/step - loss: 5.9375 - o1_loss: 0.5519 - o2_loss: 0.5487 - o3_loss: 0.6005 - o4_loss: 0.7043 - o5_loss: 0.7701 - o6_loss: 0.8597 - o7_loss: 0.9650 - o8_loss: 0.9372 - o1_acc: 0.7666 - o2_acc: 0.7591 - o3_acc: 0.7385

10000/10000 [==============================] - 21s 2ms/step - loss: 4.7378 - o1_loss: 0.4758 - o2_loss: 0.4323 - o3_loss: 0.4501 - o4_loss: 0.5268 - o5_loss: 0.6115 - o6_loss: 0.6987 - o7_loss: 0.7781 - o8_loss: 0.7645 - o1_acc: 0.8168 - o2_acc: 0.8131 - o3_acc: 0.8105 - o4_acc: 0.7753 - o5_acc: 0.7280 - o6_acc: 0.6890 - o7_acc: 0.6444 - o8_acc: 0.6352 - val_loss: 4.6281 - val_o1_loss: 0.4740 - val_o2_loss: 0.4083 - val_o3_loss: 0.4296 - val_o4_loss: 0.5213 - val_o5_loss: 0.6111 - val_o6_loss: 0.7048 - val_o7_loss: 0.7576 - val_o8_loss: 0.7213 - val_o1_acc: 0.8170 - val_o2_acc: 0.8290 - val_o3_acc: 0.8200 - val_o4_acc: 0.7730 - val_o5_acc: 0.7330 - val_o6_acc: 0.6970 - val_o7_acc: 0.6720 - val_o8_acc: 0.6510
Epoch 46/100
10000/10000 [==============================] - 21s 2ms/step - loss: 4.6714 - o1_loss: 0.4609 - o2_loss: 0.4180 - o3_loss: 0.4288 - o4_loss: 0.5102 - o5_loss: 0.5990 - o6_loss: 0.7025 - o7_loss: 0.7812 - o8_loss: 0.7707 - o1_acc: 0.8218 - o2_acc: 0.8223 - o3_acc: 0.8208

10000/10000 [==============================] - 21s 2ms/step - loss: 3.9673 - o1_loss: 0.3272 - o2_loss: 0.3272 - o3_loss: 0.3382 - o4_loss: 0.4108 - o5_loss: 0.5104 - o6_loss: 0.6235 - o7_loss: 0.7137 - o8_loss: 0.7162 - o1_acc: 0.8863 - o2_acc: 0.8782 - o3_acc: 0.8675 - o4_acc: 0.8337 - o5_acc: 0.7796 - o6_acc: 0.7247 - o7_acc: 0.6828 - o8_acc: 0.6563 - val_loss: 3.8785 - val_o1_loss: 0.3402 - val_o2_loss: 0.3110 - val_o3_loss: 0.3335 - val_o4_loss: 0.4064 - val_o5_loss: 0.4938 - val_o6_loss: 0.6178 - val_o7_loss: 0.7128 - val_o8_loss: 0.6630 - val_o1_acc: 0.8740 - val_o2_acc: 0.8860 - val_o3_acc: 0.8620 - val_o4_acc: 0.8410 - val_o5_acc: 0.7780 - val_o6_acc: 0.7410 - val_o7_acc: 0.6910 - val_o8_acc: 0.6750
Epoch 57/100
10000/10000 [==============================] - 21s 2ms/step - loss: 3.9450 - o1_loss: 0.3113 - o2_loss: 0.3152 - o3_loss: 0.3300 - o4_loss: 0.4007 - o5_loss: 0.5085 - o6_loss: 0.6275 - o7_loss: 0.7247 - o8_loss: 0.7271 - o1_acc: 0.8934 - o2_acc: 0.8891 - o3_acc: 0.8728

10000/10000 [==============================] - 21s 2ms/step - loss: 3.5002 - o1_loss: 0.2058 - o2_loss: 0.2427 - o3_loss: 0.2864 - o4_loss: 0.3599 - o5_loss: 0.4656 - o6_loss: 0.5829 - o7_loss: 0.6761 - o8_loss: 0.6808 - o1_acc: 0.9405 - o2_acc: 0.9243 - o3_acc: 0.8977 - o4_acc: 0.8634 - o5_acc: 0.8067 - o6_acc: 0.7420 - o7_acc: 0.7052 - o8_acc: 0.6821 - val_loss: 3.4478 - val_o1_loss: 0.2151 - val_o2_loss: 0.2232 - val_o3_loss: 0.2603 - val_o4_loss: 0.3541 - val_o5_loss: 0.4464 - val_o6_loss: 0.5892 - val_o7_loss: 0.6820 - val_o8_loss: 0.6775 - val_o1_acc: 0.9360 - val_o2_acc: 0.9350 - val_o3_acc: 0.9130 - val_o4_acc: 0.8600 - val_o5_acc: 0.8020 - val_o6_acc: 0.7430 - val_o7_acc: 0.6990 - val_o8_acc: 0.6700
Epoch 68/100
10000/10000 [==============================] - 21s 2ms/step - loss: 3.2558 - o1_loss: 0.1891 - o2_loss: 0.2101 - o3_loss: 0.2371 - o4_loss: 0.3066 - o5_loss: 0.4242 - o6_loss: 0.5603 - o7_loss: 0.6553 - o8_loss: 0.6732 - o1_acc: 0.9487 - o2_acc: 0.9416 - o3_acc: 0.9210

10000/10000 [==============================] - 21s 2ms/step - loss: 2.7773 - o1_loss: 0.1321 - o2_loss: 0.1540 - o3_loss: 0.1835 - o4_loss: 0.2467 - o5_loss: 0.3667 - o6_loss: 0.5046 - o7_loss: 0.5888 - o8_loss: 0.6008 - o1_acc: 0.9653 - o2_acc: 0.9597 - o3_acc: 0.9452 - o4_acc: 0.9079 - o5_acc: 0.8490 - o6_acc: 0.7757 - o7_acc: 0.7501 - o8_acc: 0.7323 - val_loss: 3.6951 - val_o1_loss: 0.1679 - val_o2_loss: 0.2182 - val_o3_loss: 0.2843 - val_o4_loss: 0.3821 - val_o5_loss: 0.5235 - val_o6_loss: 0.6320 - val_o7_loss: 0.7521 - val_o8_loss: 0.7350 - val_o1_acc: 0.9530 - val_o2_acc: 0.9360 - val_o3_acc: 0.8880 - val_o4_acc: 0.8350 - val_o5_acc: 0.7760 - val_o6_acc: 0.7300 - val_o7_acc: 0.6470 - val_o8_acc: 0.6710
Epoch 79/100
10000/10000 [==============================] - 21s 2ms/step - loss: 3.0445 - o1_loss: 0.1335 - o2_loss: 0.1703 - o3_loss: 0.2083 - o4_loss: 0.2842 - o5_loss: 0.4141 - o6_loss: 0.5743 - o7_loss: 0.6288 - o8_loss: 0.6311 - o1_acc: 0.9652 - o2_acc: 0.9534 - o3_acc: 0.9307

10000/10000 [==============================] - 21s 2ms/step - loss: 2.0335 - o1_loss: 0.0930 - o2_loss: 0.1125 - o3_loss: 0.1352 - o4_loss: 0.1885 - o5_loss: 0.2975 - o6_loss: 0.4195 - o7_loss: 0.4324 - o8_loss: 0.3548 - o1_acc: 0.9786 - o2_acc: 0.9735 - o3_acc: 0.9589 - o4_acc: 0.9337 - o5_acc: 0.8769 - o6_acc: 0.8192 - o7_acc: 0.8249 - o8_acc: 0.8601 - val_loss: 2.0568 - val_o1_loss: 0.1119 - val_o2_loss: 0.1368 - val_o3_loss: 0.1694 - val_o4_loss: 0.2021 - val_o5_loss: 0.3107 - val_o6_loss: 0.3981 - val_o7_loss: 0.4048 - val_o8_loss: 0.3231 - val_o1_acc: 0.9720 - val_o2_acc: 0.9640 - val_o3_acc: 0.9330 - val_o4_acc: 0.9280 - val_o5_acc: 0.8620 - val_o6_acc: 0.8460 - val_o7_acc: 0.8510 - val_o8_acc: 0.8730
Epoch 90/100
10000/10000 [==============================] - 21s 2ms/step - loss: 2.0050 - o1_loss: 0.0906 - o2_loss: 0.1089 - o3_loss: 0.1302 - o4_loss: 0.1851 - o5_loss: 0.2936 - o6_loss: 0.4167 - o7_loss: 0.4304 - o8_loss: 0.3494 - o1_acc: 0.9778 - o2_acc: 0.9733 - o3_acc: 0.9605

10000/10000 [==============================] - 22s 2ms/step - loss: 1.3875 - o1_loss: 0.0663 - o2_loss: 0.0831 - o3_loss: 0.0988 - o4_loss: 0.1414 - o5_loss: 0.2231 - o6_loss: 0.3080 - o7_loss: 0.2781 - o8_loss: 0.1886 - o1_acc: 0.9857 - o2_acc: 0.9794 - o3_acc: 0.9697 - o4_acc: 0.9529 - o5_acc: 0.9102 - o6_acc: 0.8678 - o7_acc: 0.8931 - o8_acc: 0.9411 - val_loss: 1.5990 - val_o1_loss: 0.0728 - val_o2_loss: 0.0943 - val_o3_loss: 0.1168 - val_o4_loss: 0.1609 - val_o5_loss: 0.2545 - val_o6_loss: 0.3555 - val_o7_loss: 0.3037 - val_o8_loss: 0.2405 - val_o1_acc: 0.9840 - val_o2_acc: 0.9710 - val_o3_acc: 0.9650 - val_o4_acc: 0.9450 - val_o5_acc: 0.8890 - val_o6_acc: 0.8470 - val_o7_acc: 0.8820 - val_o8_acc: 0.9200
